# Abstract

We provide a calculator that computes the adjoint action in the universal enveloping algebra of the Lie superalgebra $\mathcal{K}_{1|1}$ of vector fields on the superline. It is not general purpose and has been tailored to this specific Lie superalgebra. Nonetheless, we anticipate that the overall strategy of computation can be adjusted to other Lie algebras and superalgebras with relative ease. We use the standard Virasoro-like basis for $\mathcal{K}_{1|1}$. The basis elements are declared as non-commutative symbols via  Sympy. In the universal enveloping algebra we use a Poincaré-Birkhoff Witt (PBW) basis with a fixed ordering on the monomials who correspond to products (via usual multiplication) of the non-commutative basis symbols. The derivations are computed recursively.

# Introduction

We begin with a very brief review of the technical background. This notebook is mostly to demonstrate some coding proficiency. Mathematicians are invited to consult the references at the end of the document for a more precise introduction.

The Lie superalgebra of polynomial vector fields on the superline with even coordinate $x$ and odd coordinate $\xi$ is $$\text{Vec}(\mathbb{R}) = \text{span}_{\mathbb{C}[x, \xi]}\big\{\partial_x, \partial_x\big\}.$$ The subspace of contact vector fields is $\mathcal{K}_{1|1}$. A basis for this subspace is $\{e_{-1}, e_{-1/2}, e_{0}, e_{1/2}, e_{1},e_{3/2}, \ldots\}$ where $e_{n-1} = x^n\partial_x + \frac{n}{2}\xi\partial_\xi$ and $e_{n -1/2} = x^n(\partial_\xi + \xi\partial_x)$ for integral $n$.

The multiplication associated to this Lie superalgebra is denoted by a bracket $[\cdot, \cdot]$, and often referred to as the "adjoint" action. It is defined as follows: 

$$ [e_n, e_m] = (m-n)e_{n+m} \text{ if } n, m \in \mathbb{N}-1, \qquad [e_n, e_m] = (m - n/2)e_{n+m} \text{ if } n \in \mathbb{N}-1,  \text{ m } \in \mathbb{N}-1/2, \qquad [e_n, e_m] = 2e_{n+m}.$$

For example, $[e_1, e_2] = e_3$.

The universal enveloping algebra $\mathfrak{U}$ of $\mathcal{K}_{1|1}$ may be thought of as polynomials in the above basis elements. It is an associative unital non-commutative algebra. The adjoint action of $\mathcal{K}_{1|1}$ then extends to an action on $\mathfrak{U}$. For $\Theta_1, \Theta_2 \in \mathfrak{U}$, we have 

$$[e_n, \Theta_1\Theta_2] = [e_n, \Theta_1]\Theta_2 + \Theta_1[e_n, \Theta_2]$$

which then yields an action of $\mathfrak{U}$ on itself. For reference, one says the adjoint operation is a "derivation". (In fact, any linear map satisfying the so-called 'Liebniz property' as shown here is a derivation). For example:

$$
\begin{align*}
[e_1e_2, e_3e_4] 
&=\Big[e_1, [e_2, e_3]e_4 +  e_3[e_2, e_4]\Big]\\[1em]
&= \Big[e_1, e_5e_4 + 2 e_3e_6\Big] \\[1em]
&=[e_1, e_5]e_4 + e_5[e_1, e_4] + 2 [e_1, e_3]e_6 + 2 e_3[e_1, e_6] \\[1em]
&=4e_6e_4 + 3e_5e_5 + 4e_4e_6 + 10e_3e_7
\end{align*}$$

As stated previously, $\mathfrak{U}$ is non-commutative. Thus, the elements $4e_6e_4$ and $4e_4e_6$ may not be combined. One must use the rule $e_4e_6 = e_6e_4 + [e_4, e_6]$. 

Naturally, computations in $\mathfrak{U}$ are quite time-consuming and very error prone. On the other hand, they are a relatively straightforward recursive process with a simple base-case. The below code performs a shift on the indices (which have been labeled with subscripts as half-integers starting at $-1$) to be in bijection with the naturals. Thus, $e_0$ in the code below corresponds to the basis element $e_{-1}$, and $e_1$ corresponds to the basis element $e_{-1/2}$ and so on.

In [25]:
#display options
from IPython.display import display, HTML, Math, Latex
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>.container { width:99% !important; }</style>"))
import os

 
import sympy as sp
from sympy import *
import functools 

e = sp.symbols('e0:1000', commutative=False)

basis = list(e)

def write(x):
    return expand( standardize( expand( x )))

def getweight(x):
    return float((basis.index(x))/2 - 1)

def getindex(x):
    return int(2*(x + 1))

def getparity(x):
    L = unpack(x)
    parity=0
    for term in L:
        parity += int(basis.index(term))
    return int(parity%2)

def unpack(x):#check for multiplicity and write powers as distinct elements in list
    if type(x).__name__ == 'Mul': #check if passed argument is product e.g. e[1]*e[2]
        L = list(x.args)
        for term in L:
            if type(term).__name__ == 'Pow':#check if passed argument is product that contains powers e.g. e[1]*e[2]**2
                J = list(term.args) #returns [vector, power]
                power = J[1]
                base = J[0]
                firstL = L[:L.index(term)] #splitting the list to insert multiplicity many elements
                secondL = L[L.index(term) + 1: ]
                for i in range(power):
                    firstL.append(base)
                L = firstL + secondL
        return L
            
    if type(x).__name__ == 'Pow':#check if passed argument is just a power itself e.g. e[1]**2
        L = list(x.args) #returns [vector, power]
        base = L[0]
        power = L[1]
        L=[]
        for i in range(power):
            L += unpack(base)
        return L
            
    if type(x).__name__ == 'Symbol':#check if passed argument is single basis vector e.g. e[1]
        return [x]
    

def bracket(x,y):
    if x.is_number or y.is_number:
        return 0
    
    #bilinearity 
    if type(x).__name__ == 'Add':#split across sums on x
        L = list(x.args)
        J = []
        for term in L:
            J.append( bracket(term, y) )
        return sum(J)
        
    if type(y).__name__ == 'Add':#split across sums on y
        L = list(y.args)
        J = []
        for term in L:
            J.append( bracket(x, term) )
        return sum(J)
    
    if type(x).__name__ == 'Mul':    #pull out x coefficients
        L = list(x.args)
        if L[0].is_number:
            coeff = L[0]
            del L[0]
            return coeff*bracket( prod(L) , y )

    if type(y).__name__ == 'Mul':    #pull out y coefficients
        L = list(y.args)
        if L[0].is_number:
            coeff = L[0]
            del L[0]
            return coeff*bracket( x , prod(L) )
    
    
    #ad homomorphism property
    unpackedx = unpack(x)
    if len(unpackedx) > 1:#reducing product
        xreductionprod = prod(unpackedx[:-1])
        return bracket(xreductionprod, bracket(unpackedx[-1], y))
    
    #super Leibniz
    unpackedy = unpack(y)
    if len(unpackedy) > 1:
        yreductionprod = prod(unpackedy[1:])
        leibniz1 = bracket(x, unpackedy[0]) * yreductionprod
        leibniz2 = unpackedy[0] * bracket(x, yreductionprod)
        derivationParity = int((-1)**(getparity(x) * getparity(unpackedy[0])))
        return leibniz1 + derivationParity*leibniz2
        
    
    #base case
    weightx = getweight(x)
    weighty = getweight(y)
    combinedweight = weightx + weighty
    combinedindex = getindex(combinedweight)
    
    if weightx.is_integer() and weighty.is_integer():
        z = (weighty - weightx)*e[combinedindex]
    if weightx.is_integer() and not weighty.is_integer():
        z = (weighty - weightx/2)*e[combinedindex]
    if not weightx.is_integer() and weighty.is_integer():
        z = -1*(weightx - weighty/2)*e[combinedindex]
    if not weightx.is_integer() and not weighty.is_integer():
        z = 2*e[combinedindex]
    return z

def standardize(x):
    if type(x).__name__ == 'Symbol' or x.is_number:
        return x
    
    if type(x).__name__ == 'Add':#split across sums on x
        L = list(x.args)
        J = []
        for term in L:
            J.append( standardize(term) )
        return sum(J)

    if type(x).__name__ == 'Mul':    #pull out x coefficients
        L = list(x.args)
        if L[0].is_number:
            coeff = L[0]
            del L[0]
            return coeff*standardize( prod(L) )
    
    L = unpack(x)
    ordercheck = 0
    indexforswap = 0
    
    for i in range(len(L)-1):#store index of point for swap, exit loop and then call swap with bracket   
        if basis.index(L[i]) > basis.index(L[i+1]):
            indexforswap = i
            ordercheck+=1
            break
        
        if basis.index(L[i]) == basis.index(L[i+1]) and basis.index(L[i])%2 == 1:
            L[i] = .5*bracket(L[i], L[i+1])
            del L[i+1]
            return standardize( prod(L) )
            
    if ordercheck == 0: 
        return x
    
    if ordercheck != 0:
        firstJ= []
        secondJ = []
        for j in L[:i]:
            firstJ.append(j)
        firstJ.append(bracket(L[i], L[i+1])) #insert bracket into new list
            
        if indexforswap + 2 < len(L):
            for j in L[indexforswap+2:]:
                secondJ.append(j) #get second half of list
                    
        J = firstJ + secondJ
        parity = int( (-1)**( getparity(L[indexforswap] ) * getparity( L[indexforswap+1]) ) )
        L[indexforswap], L[indexforswap+1] = L[indexforswap+1], L[indexforswap] #finally swap

        return parity*standardize( prod(L) ) + standardize( prod(J) ) #run through again to reorder



#######################################################################################################################################################################
#Defining some commonly used objects.

#Step operators
S3half = (e[2] - 1)*e[5] - e[4]*e[3] #weight 3/2 step operator
S2 = (4*e[2] - 2)*e[6] - 3*e[4]**2 - 3*e[3]*e[5] #weight 2 step operator 
S5half = (2*e[2] - 3)*(2*e[2] - 1)*e[7] - 2*(2*e[2] - 3)*e[3]*e[6] - 3*(2*e[2]-3)*e[4]*e[5] - 6*e[4]*e[4]*e[3] #weight 5/2 step operator

#Quadratic Terms
Qs = e[2]**2 + .5*e[2] + .5*e[1]*e[3] - e[0]*e[4] #osp Casimir
Q = e[2]**2 - e[2] - e[4]*e[0] #sl2 Casimir
Lambda = e[2] - e[3]*e[1]
T = Lambda - 1/4 #ghost
LWVn1half = bracket(e[1], T) #lowest weight vector of weight -1/2
Te2 = bracket(e[6], T) #ad(e_2)(ghost)
Qse2 = bracket(e[6], Qs)#ad(e_2)(Casimir of osp)

#Climbing the F_0 in S^2 starting with Z_{1/2}, which drops to Casimir under e_{-1/2}
Z1half = e[4]*e[1] - 2*e[2]*e[3] + e[0]*e[5] 
Z1 = bracket(e[3], Z1half)
Z3half = bracket(e[3], Z1)
Z2 = bracket(e[3], Z3half)

#Cubic Terms
Y0 = expand(4*Qs*(e[2] - 1/4) + .5*Z1half*e[1] + Z1*e[0])#LWV of weight 0 in S^3
Yn1half = expand(2*Qs*e[1] + Z1half*e[0]) #LWV of weight -1/2 in S^3
#######################################################################################################################################################################

In [26]:
a = write(bracket(S3half, Z1half))
b = write(bracket(e[3], Z1half))
c = write(bracket(e[3], b))
d = write(bracket(e[3], c))
a - d
write(-8*bracket(e[6], T))

#This shows that T^(e_2) = ad(S3half)(Z1half) - ad(e_{1/2}^3)(Z1half). So T^(e_2) is in the ideal generated by Z1half. 

2.0*e0*e8 + 3.0*e1*e7 - 4.0*e2*e6 - 11.0*e3*e5 + 2.0*e4**2 + 8.0*e6 - (2.0*e0*e8 - 1.0*e1*e7 - 4.0*e2*e6 + 1.0*e3*e5 + 2.0*e4**2)

4.0*e1*e7 - 12.0*e3*e5 + 8.0*e6

In [27]:

#Example calculation showing that Z_{1/2} drops to a multiple of Casimir of osp under ad(e_{-1/2}) in S^2
b = bracket(e[1], Z1half)
sb = simplify(standardize(b))
standqs = standardize(Qs)
standqs
sb


#Example calculation showing that Te2 = ad(e_2)(T) straddles the F_{3/2} and F_2 in S^2.
simplify(standardize(bracket(e[1], bracket(e[1], Te2))))

#Example calculation verifying that the weight 1 vector dropping into the F_-1/2 of S^2 is ( T^{e_-1/2} )^{e_3/2}
t1 = expand( standardize( bracket(e[5], LWVn1half)))
p0 = expand( bracket(e[3], LWVn1half) )
p1half = standardize( expand( bracket(e[3], p0) ) )
p1 = standardize( expand( bracket(e[3], p1half) ) )#gives 0

-e0*e4 + 0.5*e1*e3 + 0.5*e2 + e2**2

4*e0*e4 - 2.0*e1*e3 - 2.0*e2 - 4*e2**2

0

# Applications

